In [1]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
import math
import sympy
from matplotlib import pyplot as plt
import numpy as np

# Conectare la providerul cuantic

provider = QuantumRingsProvider(
    token='rings-200.Rt3oI0TUvGcnDzXQEX3rUcHfXZQ9b7kJ',
    name='mariusgh2308@gmail.com'
)

backend_name = "scarlet_quantum_rings"
available_backends = provider.backends(backend_name)
if backend_name in available_backends:
    backend = provider.get_backend(backend_name)
shots = 1024


In [13]:
def iqft_circuit(qc, qubits):
    """
    Add the Inverse Quantum Fourier Transform to a circuit.
    """
    for i in reversed(range(len(qubits))):
        qc.h(qubits[i])
        for j in range(i):
            qc.cp(-math.pi / (2 ** (i - j)), qubits[j], qubits[i])
    qc.barrier()

    return


def plot_histogram(counts):
    """
    Plot the histogram of the counts of a quantum circuit.
    """

    plt.figure(figsize=(10, 7))
    plt.bar(counts.keys(), counts.values())
    plt.xlabel("Quantum States")
    plt.ylabel("# Shots")
    plt.xticks(rotation=90)
    plt.show()

    return


def get_factors(N): 
    """
    Return the factors of a number N using Shor's algorithm.
    """

    # Pick a random number a < N, coprime with N
    while True:
        a = np.random.randint(2, N - 1)
        if math.gcd(a, N) == 1:
            return a, N // a # we randomly found the factors - yey!

    # Build the quantum circuit
    number_of_qubits = math.ceil(math.log2(N) * 2)
    q = QuantumRegister(number_of_qubits, 'q')
    c = ClassicalRegister(number_of_qubits // 2, 'c')
    qc = QuantumCircuit(q, c)

    # Initializations
    qc.h(range(number_of_qubits // 2))
    qc.x(number_of_qubits - 1)
    qc.barrier()

    # Calculate a^x mod N
    for i in range(number_of_qubits // 2):
        qc.cp(math.pi / (2 ** i), q[i], q[number_of_qubits - 1])
    qc.barrier()

    # Apply IQFT
    iqft_circuit(qc, [q[i] for i in range(number_of_qubits // 2)])

    # Measure
    for i in range(number_of_qubits // 2):
        qc.measure(q[i], c[i])

    # Execute the circuit
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()

    # Classical post-processing
    measured_states = sorted([int(state, 2) for state in counts.keys() if state.strip()])
    for r in measured_states:
        if r > 0 and r % 2 == 0:
            factor1 = sympy.gcd(a ** (r // 2) - 1, N)
            factor2 = sympy.gcd(a ** (r // 2) + 1, N)
            if 1 < factor1 < N:
                print(f"The factors of {N} are: {factor1} and {N // factor1}")
                return factor1, N // factor1 # Found the factors!
    
    return get_factors(N) # Try again!

In [23]:
from semiprimes import semiprimes
import numpy as np
for N in semiprimes.values():
    print(f"\nDetermining the factrs of {N}...")
    get_factors(N)

# Highest number found: 11426971
# Number of bits: 24
# Factors: 3191 and 3581
# And the search continues...


Determinarea factorilor numărului 55
Job Queued
Job Done.
Ending Job Monitor
Factorii lui 55 sunt: 5 și 11

Determinarea factorilor numărului 143
Job Queued
Job Done.
Ending Job Monitor
Factorii lui 143 sunt: 11 și 13

Determinarea factorilor numărului 899
Job Queued
Job Done.
Ending Job Monitor
Factorii lui 899 sunt: 29 și 31

Determinarea factorilor numărului 3127
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor
Factorii lui 3127 sunt: 59 și 53

Determinarea factorilor numărului 11009
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor
Factorii lui 11009 sunt: 109 și 101

Determinarea factorilor numărului 47053
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor
Job Queued
Job Done.
Ending Job Monitor

KeyboardInterrupt: 